In [24]:
import mediapipe as mp
import cv2
from dollarpy import Recognizer, Template, Point
import time
import pickle

mp_drawing = mp.solutions.drawing_utils
mp_holistic = mp.solutions.holistic


In [25]:
# List of video paths to train on the same gesture
video_files = [
    r"C:\Users\Youssef\Desktop\Test\WhatsApp Video 2024-11-02 at 5.30.18 PM.mp4",
    r"C:\Users\Youssef\Desktop\Test\WhatsApp Video 2024-11-02 at 6.01.43 PM(5).mp4",
    r"C:\Users\Youssef\Desktop\Test\WhatsApp Video 2024-11-02 at 6.01.43 PM.mp4",
    r"C:\Users\Youssef\Desktop\Test\WhatsApp Video 2024-11-03 at 7.56.43 PM.mp4",
    r"C:\Users\Youssef\Desktop\Test\WhatsApp Video 2024-11-02 at 8.09.51 PM (1).mp4",
    r"C:\Users\Youssef\Desktop\Test\WhatsApp Video 2024-11-02 at 8.09.51 PM.mp4",
    r"C:\Users\Youssef\Desktop\Test\WhatsApp Video 2024-11-02 at 8.14.23 PM.mp4",
    r"C:\Users\Youssef\Desktop\Test\WhatsApp Video 2024-11-02 at 8.10.09 PM.mp4",
    r"C:\Users\Youssef\Desktop\Test\IMG_6642.mov",
    # Add more video paths as needed
]

def getPoints(video_files, pinch_gesture=False):
    all_points = []
    for videoURL in video_files:
        cap = cv2.VideoCapture(videoURL)
        with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
            while cap.isOpened():
                ret, frame = cap.read()
                if not ret:
                    break
                
                image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                image.flags.writeable = False

                results = holistic.process(image)
                image.flags.writeable = True
                image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

                # Draw hand landmarks
                mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
                mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)

                # If landmarks are detected, process pinching gestures
                if results.right_hand_landmarks or results.left_hand_landmarks:
                    try:
                        hand_landmarks = results.right_hand_landmarks or results.left_hand_landmarks
                        index_tip = hand_landmarks.landmark[mp.solutions.hands.HandLandmark.INDEX_FINGER_TIP]
                        thumb_tip = hand_landmarks.landmark[mp.solutions.hands.HandLandmark.THUMB_TIP]
                        
                        # Calculate distance between thumb and index fingertip to detect pinch
                        distance = ((thumb_tip.x - index_tip.x)**2 + (thumb_tip.y - index_tip.y)**2) ** 0.5

                        # If pinch detected, add relevant points to the gesture points list
                        if pinch_gesture and distance < 0.05:  # Adjust threshold as needed
                            all_points.append(Point(index_tip.x, index_tip.y, 1))  # Label for pinch gesture

                    except:
                        pass

                # Display the frame
                cv2.imshow("Gesture Training", image)
                
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break

        cap.release()
        cv2.destroyAllWindows()

    return all_points


In [26]:
# Train the model and create the template for the Circle Gesture
all_points = getPoints(video_files, pinch_gesture=True)

# Create a single template for the Circle Gesture
circle_template = Template("Circle Gesture", all_points)
templates = [circle_template]

# Initialize recognizer with the single gesture template
recognizer = Recognizer(templates)

import pickle

# Save the model
model_filename = 'pinching.pkl'
with open(model_filename, 'wb') as model_file:
    pickle.dump(templates, model_file)

print(f"Model saved to {model_filename}")


Model saved to pinching.pkl


In [ ]:
def pinching_status_callback(pinching_detected):
    if pinching_detected:
        print("Pinching")
    else:
        print("Not pinching")

def getPointsRealTime(pinching_detected_callback):
    cap = cv2.VideoCapture(0)  # Use camera
    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
            
            # Flip the frame horizontally to mirror the camera feed
            frame = cv2.flip(frame, 1)
            
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False

            results = holistic.process(image)
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

            # Draw hand landmarks
            mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
            mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)

            # Check for pinch gesture
            pinching_detected = False
            if results.right_hand_landmarks or results.left_hand_landmarks:
                try:
                    hand_landmarks = results.right_hand_landmarks or results.left_hand_landmarks
                    index_tip = hand_landmarks.landmark[mp.solutions.hands.HandLandmark.INDEX_FINGER_TIP]
                    thumb_tip = hand_landmarks.landmark[mp.solutions.hands.HandLandmark.THUMB_TIP]
                    
                    # Calculate distance between thumb and index fingertip to detect pinch
                    distance = ((thumb_tip.x - index_tip.x)**2 + (thumb_tip.y - index_tip.y)**2) ** 0.5

                    if distance < 0.02:  # Adjust threshold as needed
                        pinching_detected = True
                        # Draw a green dot at the pinch location
                        pinch_x = int(index_tip.x * image.shape[1])
                        pinch_y = int(index_tip.y * image.shape[0])
                        cv2.circle(image, (pinch_x, pinch_y), 10, (0, 255, 0), -1)  # Draw green dot

                except Exception as e:
                    print(f"Error processing hand landmarks: {e}")

            # Call the callback with the current pinching status
            pinching_detected_callback(pinching_detected)

            # Display the frame
            cv2.imshow("Gesture Detection", image)
            
            if cv2.waitKey(10) & 0xFF == ord('q'):
                break

        cap.release()
        cv2.destroyAllWindows()

# Start real-time gesture detection
getPointsRealTime(pinching_status_callback)


Not pinching
Not pinching
Not pinching
Not pinching
Not pinching
Not pinching
Not pinching
Not pinching
Not pinching
Not pinching
Not pinching
Not pinching
Not pinching
Not pinching
Not pinching
Not pinching
Not pinching
Not pinching
Not pinching
Not pinching
Not pinching
Not pinching
Not pinching
Not pinching
Not pinching
Not pinching
Not pinching
Not pinching
Not pinching
Not pinching
Not pinching
Not pinching
Not pinching
Not pinching
Not pinching
Not pinching
Not pinching
Not pinching
Not pinching
Not pinching
Not pinching
Not pinching
Not pinching
Not pinching
Not pinching
Not pinching
Not pinching
Not pinching
Not pinching
Not pinching
Not pinching
Not pinching
Not pinching
Not pinching
Not pinching
Not pinching
Not pinching
Not pinching
Not pinching
Not pinching
Not pinching
Not pinching
Not pinching
Not pinching
Not pinching
Not pinching
Not pinching
Not pinching
Not pinching
Not pinching
Not pinching
Not pinching
Not pinching
Not pinching
Not pinching
Not pinching
Not pinching

KeyboardInterrupt: 

: 